<a href="https://colab.research.google.com/github/hanumantjain/agentic-ai/blob/main/LLM_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!!pip install litellm --quiet

import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [5]:
import json
import json
from typing import List
from litellm import completion

def list_files() -> List[str]:
  return os.listdir('.')

def read_file(file_name: str) -> str:
  try:
    with open(file_name, 'r') as file:
      return file.read()
  except FileNotFoundError:
    return f"Error: {file_name} not found."
  except Exception as e:
    return f"Error: {str(e)}"

tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": { "type": "object", "properties": {}, "required": [] }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory",
            "parameters": {
                "type": "object",
                "properties": { "file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]

agent_rules = [{
    'role': "system",
    "content": """
        You are an AI agent that can perform tasks by using available tools.

        If a user asks about files, documents or content, first list the files before reading them.
    """
}]

user_task = input("What would you like me to do? ")

memory =[{
    "role": "user",
    "content": user_task
}]

messages = agent_rules + memory

response = completion(
    model="openai/gpt-4o-mini",
    messages = messages,
    tools = tools,
    max_tokens = 512
)

tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
result = tool_functions[tool_name](**tool_args)

print(f"Tool name: {tool_name}")
print(f"Tool arguments: {tool_args}")
print(f"Result: {result}")

What would you like me to do? list files
Tool name: list_files
Tool arguments: {}
Result: ['.config', 'sample_data']
